In [ ]:
import pandas as pd
import numpy as np
import duckdb
import pyCLIF as pc
from tqdm import tqdm
from datetime import datetime
from tableone import TableOne, load_dataset
con = pc.load_config()

In [ ]:
cohort = pd.read_csv('../output/intermediate/study_cohort.csv')

In [ ]:
cohort['sat_delivery_pass_fail'] = cohort['sat_delivery_pass_fail'].map({0:1,1:1})
cohort['sat_screen_pass_fail'] = cohort['sat_screen_pass_fail'].map({0:1,1:1})

In [ ]:
# Ensure 'event_time' is in datetime format
cohort['event_time'] = pd.to_datetime(cohort['event_time'])
cohort['admission_dttm'] = pd.to_datetime(cohort['admission_dttm'], utc=True)
cohort['discharge_dttm'] = pd.to_datetime(cohort['discharge_dttm'], utc=True)

# Ensure the data is sorted by 'hosp_id_day_key' and 'event_time'
cohort = cohort.sort_values(by=['hospitalization_id', 'event_time']).reset_index(drop=True)

cohort['device_category_ffill'] = cohort.groupby('hospitalization_id')['device_category'].ffill()
cohort['location_category_ffill'] = cohort.groupby('hospitalization_id')['location_category'].ffill()

active_sedation_n_col = [
    'fentanyl', 'propofol', 'lorazepam', 'midazolam', 'hydromorphone', 'morphine'
]

for col in active_sedation_n_col:
    if col not in cohort.columns:
        cohort[col] = np.nan
        print(f"Column '{col}' is missing. Please check your CLIF Meds table — it might be missing, or it's okay if your site doesn't use it.")


# Fill forward the meds by hospitalization columns by 'hosp_id'
cohort[['fentanyl', 'propofol', 'lorazepam', 'midazolam', 'hydromorphone', 'morphine']] = cohort.groupby('hospitalization_id')[
    ['fentanyl', 'propofol', 'lorazepam', 'midazolam', 'hydromorphone', 'morphine']
].ffill()

# Ensure the min value is greater than 0
cohort['min_sedation_dose'] = cohort[['fentanyl', 'propofol', 'lorazepam', 'midazolam','hydromorphone','morphine']].min(axis=1, skipna=True)
cohort['min_sedation_dose_2'] = cohort[['fentanyl', 'propofol', 'lorazepam', 'midazolam', 'hydromorphone', 'morphine']].where(cohort[['fentanyl', 'propofol', 'lorazepam', 'midazolam', 'hydromorphone', 'morphine']] > 0).min(axis=1, skipna=True)
cohort['min_sedation_dose_non_ops'] = cohort[['propofol', 'lorazepam', 'midazolam']].min(axis=1, skipna=True)
cohort['min_sedation_dose_non_ops'] = cohort['min_sedation_dose_non_ops'].fillna(0)

# Fill forward the paralytic by hospitalization columns by 'hosp_id'
cohort[["cisatracurium"
        ,"vecuronium"
        ,"rocuronium"]] = cohort.groupby('hospitalization_id')[
    ["cisatracurium"
        ,"vecuronium"
        ,"rocuronium"]
].ffill()
# paralytic max to remove from consideration
cohort['max_paralytics'] = cohort[["cisatracurium"
        ,"vecuronium"
        ,"rocuronium"
        ]].max(axis=1, skipna=True).fillna(0)

# Ensure the data is sorted again by 'hosp_id_day_key' and 'event_time'
cohort = cohort.sort_values(by=['hospitalization_id', 'event_time']).reset_index(drop=True)

#### Identify eligible days

In [ ]:
def process_cohort(df):
    df = df.sort_values(by=['hospitalization_id', 'event_time']).reset_index(drop=True)
    df['device_category_ffill'] = df.groupby('hospitalization_id')['device_category'].ffill()
    df['location_category_ffill'] = df.groupby('hospitalization_id')['location_category'].ffill()
    # Ensure 'event_time' is datetime
    df['event_time'] = pd.to_datetime(df['event_time'])
   
    df['all_conditions_check'] = (
            (df['device_category_ffill'].str.lower() == 'imv') &
            (df['min_sedation_dose_2'] > 0) &
            (df['location_category_ffill'].str.lower() == 'icu') &
            (df['max_paralytics'] <= 0)
        ).astype(int)

    # Initialize result list
    result = []

    # Group by 'hospitalization_id' and 'date'
    grouped_hosp = df.groupby(['hospitalization_id', df['event_time'].dt.normalize()])

    # Use tqdm for the outer loop to show progress
    for (hosp_id, date), group in tqdm(grouped_hosp, desc='Processing Hospitalizations by Date'):
        group = group.sort_values('event_time')

        # Get the entire hospitalization data for the current hospitalization_id
        temp_df = df[df['hospitalization_id'] == hosp_id].sort_values(by=['hospitalization_id', 'event_time']).reset_index(drop=True)

        # Define start and end times for the current day
        # Start time is 10 PM of the previous day
        start_time = date - pd.Timedelta(days=1) + pd.Timedelta(hours=22)
        # End time is 6 AM of the current day
        end_time = date + pd.Timedelta(hours=6)

        # Filter data in this time window for the entire hospitalization
        mask_time = (temp_df['event_time'] >= start_time) & (temp_df['event_time'] <= end_time)
        df_time_window = temp_df[mask_time].copy()

        if df_time_window.empty:
            continue

        # Use the existing 'device_category_ffill' and 'location_category_ffill' columns
        df_time_window['all_conditions_met'] = (df_time_window['all_conditions_check']>0
        )

        # If no times where all conditions are met, skip
        if not df_time_window['all_conditions_met'].any():
            continue

        # Ensure data is sorted by 'event_time'
        df_time_window = df_time_window.sort_values('event_time').reset_index(drop=True)

        # Create a group identifier for continuous periods where conditions are met
        df_time_window['condition_met_group'] = (df_time_window['all_conditions_met'] != df_time_window['all_conditions_met'].shift()).cumsum()

        # Filter rows where all conditions are met
        df_conditions = df_time_window[df_time_window['all_conditions_met']].copy()
        if df_conditions.empty:
            continue

        # Group by 'condition_met_group' to identify continuous periods
        grouped_conditions = df_conditions.groupby('condition_met_group')

        found_four_hours = False
        for group_id, group_df in grouped_conditions:
            group_df = group_df.reset_index(drop=True)

            # Calculate the duration of each continuous period where all conditions are met
            group_df['duration'] = group_df['event_time'].diff().fillna(pd.Timedelta(seconds=0))
            group_df['cumulative_duration'] = group_df['duration'].cumsum()
            total_duration = group_df['cumulative_duration'].iloc[-1]

            if total_duration >= pd.Timedelta(hours=4):
                # Calculate the exact event_time when cumulative duration reaches four hours
                cumulative_duration = pd.Timedelta(seconds=0)
                for idx in range(len(group_df)):
                    cumulative_duration += group_df['duration'].iloc[idx]
                    if cumulative_duration >= pd.Timedelta(hours=4):
                        event_time_at_4_hours = group_df['event_time'].iloc[idx]
                        break

                # Append to result
                result.append({
                    'hospitalization_id': hosp_id,
                    'current_day_key': date,
                    'event_time_at_4_hours': event_time_at_4_hours
                })
                found_four_hours = True
                # Since we found a period of at least 4 hours continuous conditions met, we can proceed to the next day
                break  # Exit the loop over condition_met_group
        if found_four_hours:
            continue  # Proceed to the next day

    # Convert result to DataFrame for better representation
    result_df = pd.DataFrame(result)
    return result_df

result_df = process_cohort(cohort)
print('Encounter days with at least 4 hours of conditions met from 10 PM to 6 AM:', len(result_df))

In [ ]:
# Merge the result back into the cohort DataFrame
cohort = cohort.merge(result_df[['hospitalization_id', 'current_day_key', 'event_time_at_4_hours']], 
                      how='left', 
                      left_on=['hospitalization_id', cohort['event_time'].dt.normalize()], 
                      right_on=['hospitalization_id', 'current_day_key'])

# Initialize 'eligible_event' column with NaN and used for validation of exact time the event of 4 hr completed
cohort['eligible_event'] = np.nan
has_event_time = cohort['event_time_at_4_hours'].notna()
for (hosp_id, date), group in cohort[has_event_time].groupby(['hospitalization_id', cohort['event_time'].dt.normalize()]):
    event_time_at_4_hours = group['event_time_at_4_hours'].iloc[0]
    subset = cohort[(cohort['hospitalization_id'] == hosp_id) & (cohort['event_time'] >= event_time_at_4_hours)]
    if not subset.empty:
        idx = subset['event_time'].idxmin()
        cohort.loc[idx, 'eligible_event'] = 1
    else:
        subset = cohort[cohort['hospitalization_id'] == hosp_id]
        idx = subset['event_time'].idxmax()
        cohort.loc[idx, 'eligible_event'] = 1

# fix where last row should not be eligible
cohort = cohort.sort_values(['hospitalization_id', 'event_time']).reset_index(drop=True)
for hosp_id, group in cohort.groupby('hospitalization_id'):
    last_idx = group.index[-1]
    if cohort.loc[last_idx, 'eligible_event'] == 1:
        cohort.loc[last_idx, 'eligible_event'] = np.nan


# Flag all that date rows where eligible_event = 1
filtered_cohort = cohort[cohort['eligible_event'] == 1][['hosp_id_day_key', 'eligible_event']]
merged_cohort = cohort.merge(filtered_cohort, on='hosp_id_day_key', how='left', suffixes=('', '_filtered'))
merged_cohort['on_vent_and_sedation'] = merged_cohort['eligible_event_filtered'].fillna(0).astype(int)
merged_cohort = merged_cohort.drop(columns=['eligible_event_filtered'])

del filtered_cohort,result_df

In [ ]:
merged_cohort['eligible_event'].value_counts()

In [ ]:
merged_cohort[merged_cohort['on_vent_and_sedation']==1]['hosp_id_day_key'].nunique()

In [ ]:
df = merged_cohort[merged_cohort['on_vent_and_sedation']==1].sort_values(by=['hospitalization_id', 'event_time']).reset_index(drop=True)  

df['rank_sedation'] = np.nan
for hosp_id_day_key, hosp_data in tqdm(df[df['on_vent_and_sedation'] == 1].groupby('hosp_id_day_key'), desc='Processing hosp_id_day_keys'):
    zero_mask = hosp_data['min_sedation_dose'] == 0
    ranks = zero_mask.cumsum() * zero_mask
    df.loc[hosp_data.index, 'rank_sedation'] = ranks.replace(0, np.nan)


df['rank_sedation_non_ops'] = np.nan
for hosp_id_day_key, hosp_data in tqdm(df[df['on_vent_and_sedation'] == 1].groupby('hosp_id_day_key'), desc='Processing hosp_id_day_keys'):
    zero_mask = hosp_data['min_sedation_dose_non_ops'] == 0
    ranks = zero_mask.cumsum() * zero_mask
    df.loc[hosp_data.index, 'rank_sedation_non_ops'] = ranks.replace(0, np.nan)

#### SAT EHR all meds hard stop flaging

In [ ]:
df['SAT_EHR_delivery'] = np.nan
med_columns = ['fentanyl', 'propofol', 'lorazepam', 'midazolam', 'hydromorphone', 'morphine']

# Use groupby and vectorized operations for meds check
for hosp_id_day_key, hosp_data in tqdm(df[df['on_vent_and_sedation'] == 1].groupby('hosp_id_day_key'), desc='Processing hosp_id_day_keys for meds check'):
    hosp_data_sorted = hosp_data.sort_values('event_time')
    for index, row in hosp_data_sorted.iterrows():
        if not np.isnan(row['rank_sedation']):
            current_time = row['event_time']
            thirty_min_forward = hosp_data_sorted[(hosp_data_sorted['event_time'] >= current_time) &
                                                  (hosp_data_sorted['event_time'] <= current_time + pd.Timedelta(minutes=30))]
            # Check if all med_columns are either NaN or 0 and device & location categories are "imv" and "icu" in this timeframe
            if (
                 (thirty_min_forward[med_columns].isna() | (thirty_min_forward[med_columns] == 0)).all(axis=None) and
                 (thirty_min_forward['device_category_ffill'] == 'imv').all() and
                 (thirty_min_forward['location_category_ffill'] == 'icu').all()
            ):
                df.at[index, 'SAT_EHR_delivery'] = 1

#### SAT EHR all meds hard stop flaging (modified meds / non ops)

In [ ]:
df['SAT_modified_delivery'] = np.nan
med_columns = ['propofol', 'lorazepam', 'midazolam']

# Use groupby and vectorized operations for meds check
for hosp_id_day_key, hosp_data in tqdm(df[df['on_vent_and_sedation'] == 1].groupby('hosp_id_day_key'), desc='Processing hosp_id_day_keys for meds check'):
    hosp_data_sorted = hosp_data.sort_values('event_time')
    for index, row in hosp_data_sorted.iterrows():
        if not np.isnan(row['rank_sedation_non_ops']):
            current_time = row['event_time']
            thirty_min_forward = hosp_data_sorted[(hosp_data_sorted['event_time'] >= current_time) &
                                                  (hosp_data_sorted['event_time'] <= current_time + pd.Timedelta(minutes=30))]

            # Check if all med_columns are either NaN or 0 and device & location categories are "imv" and "icu" in this timeframe
            if (
                (thirty_min_forward[med_columns].isna() | (thirty_min_forward[med_columns] == 0)).all(axis=None) and
                 (thirty_min_forward['device_category_ffill'] == 'imv').all() and
                 (thirty_min_forward['location_category_ffill'] == 'icu').all()
            ):
                df.at[index, 'SAT_modified_delivery'] = 1

#### Icu los calculation

In [ ]:
icu_los = cohort[['hospitalization_id', 'event_time', 'location_category_ffill']]
icu_los = icu_los.sort_values(by=['hospitalization_id', 'event_time']).reset_index(drop=True)

icu_los['segment'] = (icu_los['location_category_ffill'] != icu_los['location_category_ffill'].shift()).cumsum()

icu_segments = icu_los[icu_los['location_category_ffill'].str.lower() == 'icu'].groupby(
    ['hospitalization_id', 'segment']
).agg(
    location_start=('event_time', 'first'),
    location_end=('event_time', 'last')
).reset_index()

icu_segments['los_days'] = (icu_segments['location_end'] - icu_segments['location_start']).dt.total_seconds() / (24 * 3600)
icu_los_per_encounter = icu_segments[['hospitalization_id', 'los_days']]

total_icu_los_per_hosp = icu_los_per_encounter.groupby('hospitalization_id', as_index=False).agg(
    ICU_LOS=('los_days', 'sum')
)
total_icu_los_per_hosp.shape

#### last dishcharge hosptial_id

In [ ]:
last_hosp = cohort[['hospitalization_id', 'event_time', 'hospital_id']]

last_hosp = last_hosp.sort_values(by=['hospitalization_id','event_time'], ascending=False).groupby(
    ['hospitalization_id'], as_index=False
).agg(({'hospital_id': 'first'})).reset_index(drop=True)
last_hosp.shape

#### Table one df 

In [ ]:
main = df[['patient_id', 'hospitalization_id', 'admission_dttm', 'discharge_dttm',
       'age_at_admission', 'discharge_category', 'sex_category',
       'race_category', 'ethnicity_category','hosp_id_day_key']].drop_duplicates()
main.shape

In [ ]:
main = pd.merge(main, total_icu_los_per_hosp, on='hospitalization_id', how='left')
main = pd.merge(main, last_hosp, on='hospitalization_id', how='left')
main.shape

In [ ]:
# Columns to group by
group_cols = [
 'hosp_id_day_key'
]

max_cols = ['sat_screen_pass_fail','sat_delivery_pass_fail','SAT_EHR_delivery', 'SAT_modified_delivery', 'eligible_event']
agg_dict = {col: 'max' for col in max_cols}

df_grouped = df.groupby(group_cols).agg(agg_dict).reset_index()

df_grouped = df_grouped.sort_values('hosp_id_day_key').reset_index(drop=True)

df_grouped['sat_flowsheet_delivery_flag'] = np.where(
    (
        (df_grouped['sat_screen_pass_fail'] == 1) |
        (df_grouped['sat_delivery_pass_fail'] == 1)
    ) &
    (df_grouped['eligible_event'] == 1),
    1,  # Flag is set to 1 (True) if conditions are met
    np.nan   # Flag nan
)

final_df = main.merge(df_grouped, on='hosp_id_day_key', how='inner')
final_df.shape

In [ ]:
for x in ['sat_delivery_pass_fail', 'sat_screen_pass_fail', 'SAT_EHR_delivery',
       'SAT_modified_delivery', 'eligible_event',
       'sat_flowsheet_delivery_flag']:
    print(final_df[x].value_counts())
    print()

In [ ]:
final_df.to_csv('../output/intermediate/final_df.csv', index=False)

#### table one print

In [ ]:
categorical_columns = ['sex_category', 'race_category', 'ethnicity_category','discharge_category']
non_categorical_columns = ['age_at_admission',  'ICU_LOS', 'Inpatient_LOS']

final_df['admission_dttm'] = pd.to_datetime(final_df['admission_dttm'],utc=True)
final_df['discharge_dttm'] = pd.to_datetime(final_df['discharge_dttm'],utc=True)

In [ ]:
### SAT FLAG Table 1


sat_flow_t1 = final_df[final_df['sat_flowsheet_delivery_flag'] == 1][[ 'hospitalization_id', 'admission_dttm', 'discharge_dttm', 'age_at_admission', 'discharge_category', 'sex_category','race_category', 'ethnicity_category','ICU_LOS']].drop_duplicates()
sat_flow_t1['Inpatient_LOS'] = (sat_flow_t1['discharge_dttm'] - sat_flow_t1['admission_dttm']).dt.total_seconds() / (24 * 3600)

if len(sat_flow_t1)>1:
    table1 = TableOne(sat_flow_t1, categorical=categorical_columns, nonnormal=non_categorical_columns, columns=categorical_columns+non_categorical_columns )

    table1.to_csv(f'../output/final/table1_sat_flowhseet_{pc.helper["site_name"]}_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv')
    print(table1)

In [ ]:
### SAT EHR FLAG Table 1

sat_ehr_t1 = final_df[(final_df['SAT_EHR_delivery'] == 1) | (final_df['SAT_modified_delivery'] == 1)][[ 'hospitalization_id', 'admission_dttm', 'discharge_dttm', 'age_at_admission', 'discharge_category', 'sex_category','race_category', 'ethnicity_category','ICU_LOS']].drop_duplicates()
sat_ehr_t1['Inpatient_LOS'] = (sat_ehr_t1['discharge_dttm'] - sat_ehr_t1['admission_dttm']).dt.total_seconds() / (24 * 3600)

if len(sat_ehr_t1)>1:
    table2 = TableOne(sat_ehr_t1, categorical=categorical_columns, nonnormal=non_categorical_columns, columns=categorical_columns+non_categorical_columns )

    table2.to_csv(f'../output/final/table1_sat_ehr_{pc.helper["site_name"]}_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv')
    print(table2)

In [ ]:
### all Table 1

all_t1 = final_df[[ 'hospitalization_id', 'admission_dttm', 'discharge_dttm', 'age_at_admission', 'discharge_category', 'sex_category','race_category', 'ethnicity_category','ICU_LOS']].drop_duplicates()
all_t1['Inpatient_LOS'] = (all_t1['discharge_dttm'] - all_t1['admission_dttm']).dt.total_seconds() / (24 * 3600)

if len(all_t1)>1:
    table3 = TableOne(all_t1, categorical=categorical_columns, nonnormal=non_categorical_columns, columns=categorical_columns+non_categorical_columns )

    table3.to_csv(f'../output/final/table1_all_t1_{pc.helper["site_name"]}_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv')
    print(table3)

#### Per hospital stats

In [ ]:
# Initialize an empty list to store each hospital's data
data_list = []

# Iterate over unique hospital IDs as strings
for x in final_df['hospital_id'].astype(str).unique():
    # Calculate counts based on specific conditions
    eligible_event_count = final_df[(final_df['eligible_event'] == 1) & (final_df['hospital_id'].astype(str) == x)].shape[0]
    sat_flowsheet_delivery_flag_count = final_df[(final_df['sat_flowsheet_delivery_flag'] == 1) & (final_df['hospital_id'].astype(str) == x)].shape[0]
    SAT_modified_delivery_count = final_df[(final_df['SAT_modified_delivery'] == 1) & (final_df['hospital_id'].astype(str) == x)].shape[0]
    SAT_EHR_delivery_count = final_df[(final_df['SAT_EHR_delivery'] == 1) & (final_df['hospital_id'].astype(str) == x)].shape[0]

    SAT_EHR_uni_pats = final_df[(final_df['SAT_EHR_delivery'] == 1) & (final_df['hospital_id'].astype(str) == x)]['patient_id'].nunique()
    SAT_EHR_hosp = final_df[(final_df['SAT_EHR_delivery'] == 1) & (final_df['hospital_id'].astype(str) == x)]['hospitalization_id'].nunique()

    SAT_modified_uni_pats = final_df[(final_df['SAT_modified_delivery'] == 1) & (final_df['hospital_id'].astype(str) == x)]['patient_id'].nunique()
    SAT_modified_hosp = final_df[(final_df['SAT_modified_delivery'] == 1) & (final_df['hospital_id'].astype(str) == x)]['hospitalization_id'].nunique()

    SAT_EHR_modified_uni_pats = final_df[((final_df['SAT_EHR_delivery'] == 1) | (final_df['SAT_modified_delivery'] == 1)) & (final_df['hospital_id'].astype(str) == x)]['patient_id'].nunique()
    SAT_EHR_modified_hosp = final_df[((final_df['SAT_EHR_delivery'] == 1) | (final_df['SAT_modified_delivery'] == 1)) & (final_df['hospital_id'].astype(str) == x)]['hospitalization_id'].nunique()

    SAT_flowsheet_uni_pats = final_df[(final_df['sat_flowsheet_delivery_flag'] == 1) & (final_df['hospital_id'].astype(str) == x)]['patient_id'].nunique()
    SAT_flowsheet_hosp = final_df[(final_df['sat_flowsheet_delivery_flag'] == 1) & (final_df['hospital_id'].astype(str) == x)]['hospitalization_id'].nunique()

    # Safeguard against division by zero
    if eligible_event_count > 0:
        percent_sat_flowsheet_delivery_flag = (sat_flowsheet_delivery_flag_count / eligible_event_count) * 100
        percent_SAT_modified_delivery = (SAT_modified_delivery_count / eligible_event_count) * 100
        percent_SAT_EHR_delivery = (SAT_EHR_delivery_count / eligible_event_count) * 100
    else:
        percent_sat_flowsheet_delivery_flag = 0
        percent_SAT_modified_delivery = 0
        percent_SAT_EHR_delivery = 0

    # Append the data for this hospital to the list
    data_list.append({
        'Site_Name_Hosp': pc.helper["site_name"] + '_' + x,  
        '%_of_SAT_flowsheet_delivery_flag': percent_sat_flowsheet_delivery_flag,
        '%_of_SAT_modified_delivery': percent_SAT_modified_delivery,
        '%_of_SAT_EHR_delivery': percent_SAT_EHR_delivery,
        'eligible_event_count': eligible_event_count,
        'sat_flowsheet_delivery_flag_count': sat_flowsheet_delivery_flag_count,
        'SAT_modified_delivery_count': SAT_modified_delivery_count,
        'SAT_EHR_delivery_count': SAT_EHR_delivery_count,

        'SAT_EHR_unique_patients': SAT_EHR_uni_pats,
        'SAT_EHR_unique_hospitalizations': SAT_EHR_hosp,
        'SAT_modified_unique_patients': SAT_modified_uni_pats,
        'SAT_modified_unique_hospitalizations': SAT_modified_hosp,
        'SAT_EHR_modified_unique_patients': SAT_EHR_modified_uni_pats,
        'SAT_EHR_modified_unique_hospitalizations': SAT_EHR_modified_hosp,
        'SAT_flowsheet_unique_patients': SAT_flowsheet_uni_pats, 
        'SAT_flowsheet_unique_hospitalizations': SAT_flowsheet_hosp   
    })

# Create a DataFrame from the list
final_data_df = pd.DataFrame(data_list)
final_data_df.to_csv(f'../output/final/sat_stats_{pc.helper["site_name"]}_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv',index=False)
# Display the final DataFrame
final_data_df.T

#### Thank You!!! keep latest timestamp files and upload to box :)